In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import pathlib
import json
import numpy as np
from ophys_etl.modules.decrosstalk.ophys_plane import OphysROI, OphysMovie
from ophys_etl.modules.segmentation.qc_utils.roi_utils import add_roi_boundaries_to_img

In [ ]:
data_dir = pathlib.Path('/Users/scott.daniel/Pika/deep_interpolation/data')
assert data_dir.is_dir()

In [ ]:
movie_path = data_dir / 'denoised.h5'
roi_path = data_dir / 'denoised_segmentation_output.json'
assert movie_path.is_file()
assert roi_path.is_file()

In [ ]:
with open(roi_path, 'rb') as in_file:
    raw_roi = json.load(in_file)

roi_list = []
for i_roi, roi_data in enumerate(raw_roi):
    roi = OphysROI(height=roi_data['height'],
                   width=roi_data['width'],
                   mask_matrix=roi_data['mask_matrix'],
                   x0=roi_data['x'],
                   y0=roi_data['y'],
                   roi_id=i_roi, valid_roi=True)
    roi_list.append(roi)

In [ ]:
ophys_movie = OphysMovie(str(movie_path.absolute()),
                         motion_border={'x0':0, 'x1':0, 'y0':0, 'y1':0})

In [ ]:
%%time
ophys_movie.load_movie_data()

In [ ]:
%%time
max_img = ophys_movie._data.max(axis=0)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(max_img, cmap='gray')

In [ ]:
%%time
max_img_rgb = ophys_movie.get_max_rgb()

In [ ]:
max_img_rgb.min()

In [ ]:
max_img_rgb.max()

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(max_img_rgb)

In [ ]:
new_img = add_roi_boundaries_to_img(max_img_rgb,
                                    roi_list=roi_list,
                                    alpha=0.5)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(new_img)